**Trial**

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
!pip -qqq install transformers datasets wandb

In [ ]:
from transformers import BertModel, BertTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


In [ ]:
device = torch.device('cuda')

In [ ]:
from google.colab import drive
import pandas as pd
# Mount Google Drive
drive.mount('/content/drive')

# Now, you can access your data in Google Drive
# For example, if your data is in a folder named 'data' in the root of your Google Drive:
data_path = '/content/drive/My Drive/Test Suicide/reddit_suicide_detection_final_clean.csv'
data = pd.read_csv(data_path)
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text,class,cleaned_text
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,sex wife threaten suicide recently leave wife ...
1,Am I weird I don't get affected by compliments...,non-suicide,weird not affect compliment come know girl but...
2,Finally 2020 is almost over... So I can never ...,non-suicide,finally never hear bad year swear fucking god ...
3,i need helpjust help me im crying so hard,suicide,need help help cry hard
4,It ends tonight.I can’t do it anymore. \nI quit.,suicide,end tonight not anymore quit


In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#using pretrained model, we get rid of the embedding we created earlier
data = data.drop('cleaned_text', axis=1)
#convert the classes to 0: non suicide and 1: suicide
class_mapping = {'suicide': 1, 'non-suicide': 0}
# Replace to map class labels based on the dictionary
data['class'] = data['class'].replace(class_mapping)

data.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,It ends tonight.I can’t do it anymore. \nI quit.,1


In [ ]:
#Splitting the dataset
from sklearn.model_selection import train_test_split

# Splitting df into features and labels
X = data.drop('class', axis=1)  # This is your features matrix
y = data['class']  # These are your labels

# Splitting the data into training, validation, and testing sets
# First, split into training and a temporary set (combining validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Then split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


# Extract the text column from the DataFrame as a list of strings
X_train= X_train['text'].tolist()
X_val= X_val['text'].tolist()
X_test = X_test['text'].tolist()

y_train = y_train.tolist()
y_val = y_val.tolist()
y_test = y_test.tolist()


In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
#custom metrics for the trainer
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer and training arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.268100
1000,0.241900
1500,0.193900
2000,0.176400
2500,0.201800
3000,0.176500
3500,0.172900
4000,0.175500
4500,0.164700
5000,0.164800


TrainOutput(global_step=17418, training_loss=0.14353214140833007, metrics={'train_runtime': 4302.4963, 'train_samples_per_second': 32.386, 'train_steps_per_second': 4.048, 'total_flos': 3.66618944538624e+16, 'train_loss': 0.14353214140833007, 'epoch': 1.0})

In [15]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.09383789449930191,
 'eval_accuracy': 0.9779525750703336,
 'eval_precision': 0.9692239072256913,
 'eval_recall': 0.9735663082437276,
 'eval_f1': 0.9713902548055431,
 'eval_runtime': 172.4758,
 'eval_samples_per_second': 100.982,
 'eval_steps_per_second': 12.628,
 'epoch': 1.0}

In [16]:
trainer.save_model('final_bert_model')
tokenizer.save_pretrained('final_tokenizer')

('final_tokenizer/tokenizer_config.json',
 'final_tokenizer/special_tokens_map.json',
 'final_tokenizer/vocab.txt',
 'final_tokenizer/added_tokens.json')

In [17]:
test_dataset = Dataset(X_test_tokenized, y_test)

In [18]:
#demo inference to see if the model is accurate
text = "i am happy"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.2474, -2.1116]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9874, 0.0126]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.9873705 , 0.01262954]], dtype=float32)

In [19]:
import json
config  = json.load(open('final_bert_model/config.json'))


FileNotFoundError: [Errno 2] No such file or directory: 'Custom_bert_model/config.json'

In [22]:
saved_model = AutoModelForSequenceClassification.from_pretrained('final_bert_model')

In [23]:
saved_trainer = Trainer(
    model=saved_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
saved_trainer.predict(test_dataset).metrics

<class 'transformers.trainer_utils.EvalPrediction'>


{'test_loss': 0.1044866144657135,
 'test_accuracy': 0.97525548283385,
 'test_precision': 0.9678912783751493,
 'test_recall': 0.967746752277139,
 'test_f1': 0.9678190099305608,
 'test_runtime': 177.4048,
 'test_samples_per_second': 98.182,
 'test_steps_per_second': 12.277}

In [28]:
!zip -r final_bert_model.zip /content/final_bert_model
!zip -r final_tokenizer.zip /content/final_tokenizer


  adding: content/final_bert_model/ (stored 0%)
  adding: content/final_bert_model/training_args.bin (deflated 51%)
  adding: content/final_bert_model/model.safetensors (deflated 7%)
  adding: content/final_bert_model/config.json (deflated 49%)
updating: content/final_tokenizer/ (stored 0%)
updating: content/final_tokenizer/vocab.txt (deflated 53%)
updating: content/final_tokenizer/special_tokens_map.json (deflated 42%)
updating: content/final_tokenizer/tokenizer_config.json (deflated 75%)


In [29]:
from google.colab import files
files.download('final_bert_model.zip')
files.download('final_tokenizer.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>